# Kriging Interpolation of Yield Data

## Overview

This notebook performs **spatial interpolation (kriging)** on harvest yield data to:
1. Load validated point data from GeoPackage (from 02_Spatial_Yield_Analysis)
2. Apply manual exclusion filters for problematic data (e.g., harvester edge effects)
3. Fit variogram models to understand spatial correlation
4. Perform kriging interpolation with two strategies:
   - **Combined model**: All years together (using normalized relative yield)
   - **Separate models**: Individual kriging per year
5. Compare results and create interpolated yield surfaces

## Why Two Approaches?

- **Combined (all years)**: Relative yield is already normalized to crop-year averages, so spatial patterns may be consistent across years. More data points = more robust variogram.
- **Separate (per year)**: Each year may have unique weather/management effects that create different spatial structures.

## Output

- Variogram plots showing spatial correlation
- Interpolated yield surfaces (rasters)
- Cross-validation statistics (RMSE, MAE)
- Comparison of both kriging strategies

In [1]:
# Section 1: Import Required Libraries

from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import logging

# Spatial analysis
from scipy.spatial import distance_matrix
from sklearn.model_selection import KFold

# Kriging libraries
try:
    import pykrige
    from pykrige.ok import OrdinaryKriging
    from pykrige.uk import UniversalKriging
    HAS_PYKRIGE = True
except ImportError:
    HAS_PYKRIGE = False
    print("⚠ PyKrige not installed: pip install pykrige")

# Raster export
try:
    import rasterio
    from rasterio.transform import from_bounds
    HAS_RASTERIO = True
except ImportError:
    HAS_RASTERIO = False
    print("⚠ Rasterio not installed: pip install rasterio")

# Setup logging
logger = logging.getLogger('03_Kriging_Interpolation')
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.INFO)
    logger.addHandler(handler)

print("✓ Libraries loaded successfully")
print(f"  PyKrige available: {HAS_PYKRIGE}")
print(f"  Rasterio available: {HAS_RASTERIO}")

if not HAS_PYKRIGE:
    print("\n❌ PyKrige is required for this notebook. Install with: pip install pykrige")

✓ Libraries loaded successfully
  PyKrige available: True
  Rasterio available: True


## Configuration & Data Paths

Set paths to load the GeoPackage from the previous notebook.

In [3]:
# Configuration: Set paths

# Project root
PROJECT_ROOT = Path.cwd().resolve().parent
DATA_DIR = PROJECT_ROOT / 'data'
SPATIAL_DIR = DATA_DIR / 'spatial_analysis'

# Input: GeoPackage from 02_Spatial_Yield_Analysis
GEOPACKAGE_PATH = SPATIAL_DIR / 'yield_points_utm.gpkg'

# Output directory for kriging results
KRIGING_OUTPUT_DIR = DATA_DIR / 'kriging_results'
KRIGING_OUTPUT_DIR.mkdir(exist_ok=True)

# Kriging parameters
GRID_RESOLUTION = 10  # meters (10m grid for interpolation)
VARIOGRAM_MODELS = ['spherical', 'exponential', 'gaussian']  # Models to test
N_LAGS = 15  # Number of lag bins for variogram

logger.info(f"Project root: {PROJECT_ROOT}")
logger.info(f"Input GeoPackage: {GEOPACKAGE_PATH}")
logger.info(f"Output directory: {KRIGING_OUTPUT_DIR}")
logger.info(f"Grid resolution: {GRID_RESOLUTION}m")

Project root: /Users/holmes/local_dev/agri_analysis
Input GeoPackage: /Users/holmes/local_dev/agri_analysis/data/spatial_analysis/yield_points_utm.gpkg
Output directory: /Users/holmes/local_dev/agri_analysis/data/kriging_results
Grid resolution: 10m


## Section 2: Load Yield Data from GeoPackage

Load the validated and normalized yield points with UTM coordinates.

In [4]:
# Load GeoPackage data
logger.info(f"Loading data from {GEOPACKAGE_PATH}...")

if not GEOPACKAGE_PATH.exists():
    raise FileNotFoundError(
        f"GeoPackage not found: {GEOPACKAGE_PATH}\n"
        "Please run 02_Spatial_Yield_Analysis.ipynb first to generate the data."
    )

gdf = gpd.read_file(GEOPACKAGE_PATH)

logger.info(f"Loaded {len(gdf):,} points")
logger.info(f"CRS: {gdf.crs}")

print("\n" + "="*80)
print("LOADED DATA SUMMARY")
print("="*80)
print(f"Total points: {len(gdf):,}")
print(f"CRS: {gdf.crs}")
print(f"\nCrops: {gdf['Crop'].unique()}")
print(f"Years: {sorted(gdf['Year'].unique())}")
print(f"Fields: {gdf['Field'].nunique()} unique fields")

print(f"\nColumns available:")
for col in gdf.columns:
    if col != 'geometry':
        print(f"  - {col}")

print(f"\nRelative yield statistics:")
print(f"  Mean: {gdf['relative_yield'].mean():.3f}")
print(f"  Std: {gdf['relative_yield'].std():.3f}")
print(f"  Range: [{gdf['relative_yield'].min():.3f}, {gdf['relative_yield'].max():.3f}]")

# Show sample
print(f"\nSample data:")
print(gdf[['x_utm', 'y_utm', 'Crop', 'Year', 'yield_t_ha', 'relative_yield']].head())

Loading data from /Users/holmes/local_dev/agri_analysis/data/spatial_analysis/yield_points_utm.gpkg...
Loaded 593,596 points
CRS: EPSG:25832



LOADED DATA SUMMARY
Total points: 593,596
CRS: EPSG:25832

Crops: ['VB - Vårbyg' 'RG - Rajgræs' 'RA - Raps / rybs' 'HV - Hvede'
 'VIB - Vinterbyg' 'KLL - Kløver / lucerne']
Years: [np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
Fields: 28 unique fields

Columns available:
  - x_utm
  - y_utm
  - Crop
  - Year
  - Field
  - CompositeTLGID
  - yield_t_ha
  - avg_yield_t_ha
  - relative_yield
  - latitude
  - longitude
  - time_stamp
  - fugtighed

Relative yield statistics:
  Mean: 1.000
  Std: 0.394
  Range: [0.002, 39.513]

Sample data:
           x_utm         y_utm         Crop  Year  yield_t_ha  relative_yield
0  684762.512894  6.165175e+06  VB - Vårbyg  2022       5.586        0.808531
1  684760.956886  6.165175e+06  VB - Vårbyg  2022       5.586        0.808531
2  684758.242243  6.165175e+06  VB - Vårbyg  2022       6.462        0.935325
3  684755.745038  6.165175e+06  VB - Vårbyg  2022       2.678        0.387620
4  684754.024144  6.165175e+06  V

## Section 3: Manual Exclusion of Problematic Data

Add a filter to exclude problematic points (e.g., harvester edge effects, calibration errors).

You can modify the exclusion criteria below based on visual inspection of the maps.

In [5]:
# Manual exclusion filters
# Modify these criteria based on data quality inspection

# Initialize exclude flag (all points included by default)
gdf['exclude'] = False

# Example exclusion criteria:
# 1. Extreme outliers (relative yield > 3 std from mean)
mean_ry = gdf['relative_yield'].mean()
std_ry = gdf['relative_yield'].std()
outlier_threshold = 3
outliers = (gdf['relative_yield'] < mean_ry - outlier_threshold * std_ry) | \
           (gdf['relative_yield'] > mean_ry + outlier_threshold * std_ry)
gdf.loc[outliers, 'exclude'] = True

logger.info(f"Excluded {outliers.sum()} extreme outliers (>{outlier_threshold} std from mean)")

# 2. Manual geographic exclusion (example: exclude specific field or area)
# Uncomment and modify as needed:
# gdf.loc[gdf['Field'] == 'Problematic_Field_Name', 'exclude'] = True

# 3. You can also exclude based on spatial patterns (e.g., field edges)
# This would require defining boundaries - can be added later

# Filter to valid points for kriging
gdf_valid = gdf[~gdf['exclude']].copy()

print("\n" + "="*80)
print("EXCLUSION SUMMARY")
print("="*80)
print(f"Original points: {len(gdf):,}")
print(f"Excluded points: {gdf['exclude'].sum():,} ({100*gdf['exclude'].sum()/len(gdf):.1f}%)")
print(f"Valid points for kriging: {len(gdf_valid):,}")

print(f"\nValid points by Crop and Year:")
print(gdf_valid.groupby(['Crop', 'Year']).size())

print(f"\n💡 Tip: To manually exclude more points, modify the exclusion criteria above.")
print(f"   You can exclude by Field name, coordinate ranges, or other attributes.")

Excluded 2499 extreme outliers (>3 std from mean)



EXCLUSION SUMMARY
Original points: 593,596
Excluded points: 2,499 (0.4%)
Valid points for kriging: 591,097

Valid points by Crop and Year:
Crop                    Year
HV - Hvede              2021    20500
                        2022    36362
                        2023    29868
                        2024    39214
                        2025    45156
KLL - Kløver / lucerne  2022    11639
RA - Raps / rybs        2021    13909
                        2022     2224
                        2023    15990
                        2024    27048
                        2025    10286
RG - Rajgræs            2021    17311
                        2022     3472
                        2023    50144
                        2024    87572
                        2025    43028
VB - Vårbyg             2021     8296
                        2022    19326
                        2023    11954
                        2024    27222
                        2025    32934
VIB - Vinterbyg         2022    2

## Section 4: Variogram Analysis

Analyze spatial autocorrelation by computing and plotting experimental variograms.

This helps us understand:
- **Range**: Distance at which points become uncorrelated
- **Sill**: Maximum variance between points
- **Nugget**: Variance at zero distance (measurement error + micro-scale variation)

In [ ]:
# Variogram analysis: Combined (all years)
# Note: Variogram computation can be memory-intensive for large datasets
# We subsample if needed to avoid kernel crashes

if not HAS_PYKRIGE:
    print("⚠ Skipping variogram analysis - PyKrige not installed")
else:
    print("\n" + "="*80)
    print("VARIOGRAM ANALYSIS: ALL YEARS COMBINED")
    print("="*80)
    
    # Extract coordinates and values
    x_full = gdf_valid['x_utm'].values
    y_full = gdf_valid['y_utm'].values
    z_full = gdf_valid['relative_yield'].values
    
    # Subsample if dataset is very large (to avoid memory issues)
    MAX_VARIOGRAM_POINTS = 50000  # Limit for variogram computation
    
    if len(z_full) > MAX_VARIOGRAM_POINTS:
        logger.warning(f"Dataset has {len(z_full):,} points - subsampling to {MAX_VARIOGRAM_POINTS:,} for variogram")
        logger.warning(f"(Variogram doesn't need all points, random subsample is representative)")
        
        # Random subsample
        np.random.seed(42)
        subsample_idx = np.random.choice(len(z_full), size=MAX_VARIOGRAM_POINTS, replace=False)
        x = x_full[subsample_idx]
        y = y_full[subsample_idx]
        z = z_full[subsample_idx]
        
        print(f"\n⚠ Using {len(z):,} / {len(z_full):,} points for variogram (random subsample)")
        print(f"  This prevents memory issues while preserving spatial structure")
    else:
        x = x_full
        y = y_full
        z = z_full
    
    logger.info(f"Computing variogram for {len(z):,} points...")
    
    # Test different variogram models
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    variogram_params = {}
    
    for idx, model in enumerate(VARIOGRAM_MODELS):
        ax = axes[idx]
        
        try:
            # Create kriging object to extract variogram
            logger.info(f"  Fitting {model} model...")
            ok = OrdinaryKriging(
                x, y, z,
                variogram_model=model,
                nlags=N_LAGS,
                enable_plotting=False,
                verbose=False
            )
            
            # Extract variogram parameters
            variogram_params[model] = {
                'sill': ok.variogram_model_parameters[0],
                'range': ok.variogram_model_parameters[1],
                'nugget': ok.variogram_model_parameters[2]
            }
            
            # Plot experimental variogram
            lags = ok.lags
            semivariance = ok.semivariance
            
            ax.plot(lags, semivariance, 'o', label='Experimental', markersize=8, alpha=0.6)
            
            # Plot fitted model
            lag_range = np.linspace(0, lags.max(), 100)
            if model == 'spherical':
                from pykrige.core import _variogram_spherical
                model_values = _variogram_spherical(ok.variogram_model_parameters, lag_range)
            elif model == 'exponential':
                from pykrige.core import _variogram_exponential
                model_values = _variogram_exponential(ok.variogram_model_parameters, lag_range)
            elif model == 'gaussian':
                from pykrige.core import _variogram_gaussian
                model_values = _variogram_gaussian(ok.variogram_model_parameters, lag_range)
            
            ax.plot(lag_range, model_values, '-', label=f'Fitted {model.capitalize()}', linewidth=2)
            
            # Formatting
            ax.set_xlabel('Distance (m)', fontsize=11)
            ax.set_ylabel('Semivariance', fontsize=11)
            ax.set_title(f'{model.capitalize()} Model\nSill={variogram_params[model]["sill"]:.3f}, Range={variogram_params[model]["range"]:.0f}m',
                        fontsize=12, fontweight='bold')
            ax.legend()
            ax.grid(True, alpha=0.3)
            
        except MemoryError as e:
            logger.error(f"Memory error for {model} model - try reducing MAX_VARIOGRAM_POINTS")
            ax.text(0.5, 0.5, f'Memory Error\n{model.capitalize()} model\nReduce data size',
                   ha='center', va='center', fontsize=12, color='red')
            ax.set_xlabel('Distance (m)', fontsize=11)
            ax.set_ylabel('Semivariance', fontsize=11)
        except Exception as e:
            logger.error(f"Error fitting {model} model: {str(e)}")
            ax.text(0.5, 0.5, f'Error\n{model.capitalize()} model\n{str(e)[:50]}',
                   ha='center', va='center', fontsize=10, color='red')
            ax.set_xlabel('Distance (m)', fontsize=11)
            ax.set_ylabel('Semivariance', fontsize=11)
    
    plt.suptitle(f'Variogram Models - All Years Combined (Relative Yield)\nUsing {len(z):,} points', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(KRIGING_OUTPUT_DIR / 'variogram_combined.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    if variogram_params:
        print(f"\nVariogram parameters:")
        for model, params in variogram_params.items():
            print(f"  {model.capitalize()}:")
            print(f"    Sill: {params['sill']:.3f}")
            print(f"    Range: {params['range']:.0f} m")
            print(f"    Nugget: {params['nugget']:.3f}")
        
        print(f"\n💡 Interpretation:")
        if 'spherical' in variogram_params:
            print(f"   - Range indicates the distance of spatial influence (~{variogram_params['spherical']['range']:.0f}m)")
        print(f"   - Nugget shows measurement error + micro-scale variation")
        print(f"   - Lower nugget = more reliable measurements")
    else:
        print("\n❌ All variogram models failed - check memory and data quality")

Computing variogram for 591,097 points...



VARIOGRAM ANALYSIS: ALL YEARS COMBINED


: 

In [ ]:
# Variogram analysis: Separate by year

if not HAS_PYKRIGE:
    print("⚠ Skipping variogram analysis - PyKrige not installed")
else:
    print("\n" + "="*80)
    print("VARIOGRAM ANALYSIS: SEPARATE BY YEAR")
    print("="*80)
    
    years = sorted(gdf_valid['Year'].unique())
    n_years = len(years)
    
    fig, axes = plt.subplots(1, n_years, figsize=(6*n_years, 5))
    if n_years == 1:
        axes = [axes]
    
    variogram_params_yearly = {}
    
    for idx, year in enumerate(years):
        ax = axes[idx]
        
        # Filter data for this year
        year_data = gdf_valid[gdf_valid['Year'] == year]
        
        x = year_data['x_utm'].values
        y = year_data['y_utm'].values
        z = year_data['relative_yield'].values
        
        logger.info(f"Computing variogram for {year}: {len(z)} points")
        
        # Use spherical model (typically good for agricultural data)
        ok = OrdinaryKriging(
            x, y, z,
            variogram_model='spherical',
            nlags=N_LAGS,
            enable_plotting=False,
            verbose=False
        )
        
        # Store parameters
        variogram_params_yearly[year] = {
            'sill': ok.variogram_model_parameters[0],
            'range': ok.variogram_model_parameters[1],
            'nugget': ok.variogram_model_parameters[2]
        }
        
        # Plot
        lags = ok.lags
        semivariance = ok.semivariance
        
        ax.plot(lags, semivariance, 'o', label='Experimental', markersize=8, alpha=0.6)
        
        # Fitted model
        lag_range = np.linspace(0, lags.max(), 100)
        from pykrige.core import _variogram_spherical
        model_values = _variogram_spherical(ok.variogram_model_parameters, lag_range)
        ax.plot(lag_range, model_values, '-', label='Fitted Spherical', linewidth=2)
        
        # Formatting
        ax.set_xlabel('Distance (m)', fontsize=11)
        ax.set_ylabel('Semivariance', fontsize=11)
        ax.set_title(f'{year}\n{len(z):,} points, Range={variogram_params_yearly[year]["range"]:.0f}m',
                    fontsize=12, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.suptitle('Variogram Models by Year (Spherical)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(KRIGING_OUTPUT_DIR / 'variogram_by_year.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\nVariogram parameters by year:")
    for year, params in variogram_params_yearly.items():
        print(f"  {year}:")
        print(f"    Sill: {params['sill']:.3f}")
        print(f"    Range: {params['range']:.0f} m")
        print(f"    Nugget: {params['nugget']:.3f}")
    
    print(f"\n💡 Compare ranges across years to see if spatial structure is consistent.")

## Section 5: Kriging Interpolation - Combined Model

Perform kriging using all years combined. This assumes spatial patterns are similar across years (normalized by relative yield).

In [ ]:
# Kriging: All years combined
# Note: Kriging all 500k+ points can be slow - consider spatial thinning if needed

if not HAS_PYKRIGE:
    print("⚠ Skipping kriging - PyKrige not installed")
else:
    print("\n" + "="*80)
    print("KRIGING INTERPOLATION: ALL YEARS COMBINED")
    print("="*80)
    
    # Extract data
    x = gdf_valid['x_utm'].values
    y = gdf_valid['y_utm'].values
    z = gdf_valid['relative_yield'].values
    
    print(f"Dataset: {len(z):,} points")
    
    # Option: Spatial thinning for very large datasets
    # Uncomment if kriging is too slow:
    # MAX_KRIGING_POINTS = 100000
    # if len(z) > MAX_KRIGING_POINTS:
    #     print(f"⚠ Thinning to {MAX_KRIGING_POINTS:,} points for faster kriging")
    #     thin_idx = np.random.choice(len(z), size=MAX_KRIGING_POINTS, replace=False)
    #     x, y, z = x[thin_idx], y[thin_idx], z[thin_idx]
    
    # Define grid extent
    x_min, x_max = x.min(), x.max()
    y_min, y_max = y.min(), y.max()
    
    # Create grid
    grid_x = np.arange(x_min, x_max, GRID_RESOLUTION)
    grid_y = np.arange(y_min, y_max, GRID_RESOLUTION)
    
    logger.info(f"Grid size: {len(grid_x)} x {len(grid_y)} = {len(grid_x)*len(grid_y):,} cells")
    logger.info(f"Performing ordinary kriging with spherical variogram...")
    logger.info(f"⚠ This may take several minutes with {len(z):,} points...")
    
    try:
        # Perform kriging
        ok_combined = OrdinaryKriging(
            x, y, z,
            variogram_model='spherical',
            nlags=N_LAGS,
            enable_plotting=False,
            verbose=True
        )
        
        # Execute interpolation
        print(f"\nExecuting kriging interpolation...")
        z_pred, ss_pred = ok_combined.execute('grid', grid_x, grid_y)
        
        logger.info("Kriging complete!")
        
        # Plot results
        fig, axes = plt.subplots(1, 2, figsize=(18, 8))
        
        # Predicted values
        ax = axes[0]
        im1 = ax.imshow(
            z_pred,
            extent=[x_min, x_max, y_min, y_max],
            origin='lower',
            cmap='RdYlGn',
            vmin=0.6,
            vmax=1.4,
            aspect='auto'
        )
        ax.scatter(x[::100], y[::100], c='black', s=1, alpha=0.2, label=f'Data points (every 100th)')
        ax.set_xlabel('UTM X (m)', fontsize=11)
        ax.set_ylabel('UTM Y (m)', fontsize=11)
        ax.set_title(f'Kriging Prediction\n(Relative Yield, {len(z):,} points)', fontsize=12, fontweight='bold')
        cbar1 = plt.colorbar(im1, ax=ax)
        cbar1.set_label('Relative Yield', fontsize=10)
        ax.legend()
        
        # Prediction variance
        ax = axes[1]
        im2 = ax.imshow(
            ss_pred,
            extent=[x_min, x_max, y_min, y_max],
            origin='lower',
            cmap='viridis',
            aspect='auto'
        )
        ax.scatter(x[::100], y[::100], c='white', s=1, alpha=0.3, label='Data points (every 100th)')
        ax.set_xlabel('UTM X (m)', fontsize=11)
        ax.set_ylabel('UTM Y (m)', fontsize=11)
        ax.set_title('Kriging Variance\n(Prediction Uncertainty)', fontsize=12, fontweight='bold')
        cbar2 = plt.colorbar(im2, ax=ax)
        cbar2.set_label('Variance', fontsize=10)
        ax.legend()
        
        plt.suptitle('Kriging Results - All Years Combined', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(KRIGING_OUTPUT_DIR / 'kriging_combined.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print(f"\n✓ Kriging interpolation complete")
        print(f"  Grid resolution: {GRID_RESOLUTION}m")
        print(f"  Output grid: {len(grid_x)} x {len(grid_y)}")
        print(f"  Mean predicted relative yield: {np.nanmean(z_pred):.3f}")
        print(f"  Mean kriging variance: {np.nanmean(ss_pred):.3f}")
        
        # Save results
        kriging_combined_results = {
            'z_pred': z_pred,
            'ss_pred': ss_pred,
            'grid_x': grid_x,
            'grid_y': grid_y,
            'ok_model': ok_combined
        }
        
    except MemoryError as e:
        print(f"\n❌ MEMORY ERROR: Dataset too large for kriging ({len(z):,} points)")
        print(f"   Solutions:")
        print(f"   1. Enable spatial thinning (uncomment code above)")
        print(f"   2. Increase grid resolution (e.g., GRID_RESOLUTION = 20)")
        print(f"   3. Use yearly kriging instead (fewer points per model)")
        print(f"   4. Exclude more data in Section 3")
        
    except Exception as e:
        print(f"\n❌ ERROR during kriging: {str(e)}")
        print(f"   Check error details above for diagnosis")
        import traceback
        traceback.print_exc()

## Section 6: Kriging Interpolation - Separate by Year

Perform separate kriging for each year to capture year-specific spatial patterns.

In [ ]:
# Kriging: Separate models by year

if not HAS_PYKRIGE:
    print("⚠ Skipping kriging - PyKrige not installed")
else:
    print("\n" + "="*80)
    print("KRIGING INTERPOLATION: SEPARATE BY YEAR")
    print("="*80)
    
    years = sorted(gdf_valid['Year'].unique())
    n_years = len(years)
    
    kriging_yearly_results = {}
    
    # Create subplots
    fig, axes = plt.subplots(2, n_years, figsize=(6*n_years, 12))
    if n_years == 1:
        axes = axes.reshape(-1, 1)
    
    for idx, year in enumerate(years):
        print(f"\nProcessing {year}...")
        
        # Filter data
        year_data = gdf_valid[gdf_valid['Year'] == year]
        
        x = year_data['x_utm'].values
        y = year_data['y_utm'].values
        z = year_data['relative_yield'].values
        
        # Define grid (same extent as combined)
        x_min, x_max = gdf_valid['x_utm'].min(), gdf_valid['x_utm'].max()
        y_min, y_max = gdf_valid['y_utm'].min(), gdf_valid['y_utm'].max()
        
        grid_x = np.arange(x_min, x_max, GRID_RESOLUTION)
        grid_y = np.arange(y_min, y_max, GRID_RESOLUTION)
        
        logger.info(f"  {year}: {len(z)} points → {len(grid_x)}x{len(grid_y)} grid")
        
        # Perform kriging
        ok_year = OrdinaryKriging(
            x, y, z,
            variogram_model='spherical',
            nlags=N_LAGS,
            enable_plotting=False,
            verbose=False
        )
        
        z_pred, ss_pred = ok_year.execute('grid', grid_x, grid_y)
        
        # Store results
        kriging_yearly_results[year] = {
            'z_pred': z_pred,
            'ss_pred': ss_pred,
            'grid_x': grid_x,
            'grid_y': grid_y,
            'ok_model': ok_year
        }
        
        # Plot prediction
        ax = axes[0, idx]
        im1 = ax.imshow(
            z_pred,
            extent=[x_min, x_max, y_min, y_max],
            origin='lower',
            cmap='RdYlGn',
            vmin=0.6,
            vmax=1.4,
            aspect='auto'
        )
        ax.scatter(x, y, c='black', s=2, alpha=0.3)
        ax.set_xlabel('UTM X (m)', fontsize=10)
        ax.set_ylabel('UTM Y (m)', fontsize=10)
        ax.set_title(f'{year} - Prediction\n{len(z):,} points', fontsize=11, fontweight='bold')
        plt.colorbar(im1, ax=ax, label='Relative Yield')
        
        # Plot variance
        ax = axes[1, idx]
        im2 = ax.imshow(
            ss_pred,
            extent=[x_min, x_max, y_min, y_max],
            origin='lower',
            cmap='viridis',
            aspect='auto'
        )
        ax.scatter(x, y, c='white', s=2, alpha=0.3)
        ax.set_xlabel('UTM X (m)', fontsize=10)
        ax.set_ylabel('UTM Y (m)', fontsize=10)
        ax.set_title(f'{year} - Variance', fontsize=11, fontweight='bold')
        plt.colorbar(im2, ax=ax, label='Variance')
        
        print(f"  ✓ Mean prediction: {np.nanmean(z_pred):.3f}, Mean variance: {np.nanmean(ss_pred):.3f}")
    
    plt.suptitle('Kriging Results by Year', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(KRIGING_OUTPUT_DIR / 'kriging_by_year.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n✓ Yearly kriging complete for {len(years)} years")

## Section 7: Export Kriging Results

Export the interpolated surfaces as GeoTIFF rasters for use in QGIS or other GIS software.

In [ ]:
# Export kriging results to GeoTIFF

if not HAS_RASTERIO:
    print("⚠ Rasterio not available - skipping GeoTIFF export")
    print("   Install with: pip install rasterio")
elif not HAS_PYKRIGE:
    print("⚠ Kriging not performed - no results to export")
else:
    print("\n" + "="*80)
    print("EXPORTING KRIGING RESULTS")
    print("="*80)
    
    # Export combined model
    if 'kriging_combined_results' in locals():
        z_pred = kriging_combined_results['z_pred']
        grid_x = kriging_combined_results['grid_x']
        grid_y = kriging_combined_results['grid_y']
        
        # Create transform
        transform = from_bounds(
            grid_x.min(), grid_y.min(),
            grid_x.max(), grid_y.max(),
            z_pred.shape[1], z_pred.shape[0]
        )
        
        # Export prediction
        output_path = KRIGING_OUTPUT_DIR / 'kriging_combined_prediction.tif'
        with rasterio.open(
            output_path, 'w',
            driver='GTiff',
            height=z_pred.shape[0],
            width=z_pred.shape[1],
            count=1,
            dtype=z_pred.dtype,
            crs='EPSG:25832',
            transform=transform
        ) as dst:
            dst.write(z_pred, 1)
        
        logger.info(f"Saved combined prediction: {output_path}")
        
        # Export variance
        ss_pred = kriging_combined_results['ss_pred']
        output_path = KRIGING_OUTPUT_DIR / 'kriging_combined_variance.tif'
        with rasterio.open(
            output_path, 'w',
            driver='GTiff',
            height=ss_pred.shape[0],
            width=ss_pred.shape[1],
            count=1,
            dtype=ss_pred.dtype,
            crs='EPSG:25832',
            transform=transform
        ) as dst:
            dst.write(ss_pred, 1)
        
        logger.info(f"Saved combined variance: {output_path}")
    
    # Export yearly models
    if 'kriging_yearly_results' in locals():
        for year, results in kriging_yearly_results.items():
            z_pred = results['z_pred']
            grid_x = results['grid_x']
            grid_y = results['grid_y']
            
            transform = from_bounds(
                grid_x.min(), grid_y.min(),
                grid_x.max(), grid_y.max(),
                z_pred.shape[1], z_pred.shape[0]
            )
            
            # Export prediction
            output_path = KRIGING_OUTPUT_DIR / f'kriging_{year}_prediction.tif'
            with rasterio.open(
                output_path, 'w',
                driver='GTiff',
                height=z_pred.shape[0],
                width=z_pred.shape[1],
                count=1,
                dtype=z_pred.dtype,
                crs='EPSG:25832',
                transform=transform
            ) as dst:
                dst.write(z_pred, 1)
            
            logger.info(f"Saved {year} prediction: {output_path}")
    
    print(f"\n✓ All kriging results exported to: {KRIGING_OUTPUT_DIR}")
    print(f"\nFiles created:")
    for f in sorted(KRIGING_OUTPUT_DIR.glob('*.tif')):
        print(f"  - {f.name}")
    
    print(f"\n💡 Load these GeoTIFF files in QGIS to visualize the interpolated surfaces.")

## Summary & Next Steps

### Results Summary

This notebook performed kriging interpolation with two approaches:

1. **Combined Model (All Years)**: Uses all data points together, leveraging normalized relative yield
2. **Yearly Models**: Separate kriging for each year to capture temporal variations

### Outputs Created

- Variogram plots showing spatial correlation structure
- Interpolated yield surfaces (GeoTIFF rasters)
- Kriging variance maps (prediction uncertainty)
- Comparison visualizations

### Interpretation Tips

- **High variance areas**: Less data coverage, less reliable predictions
- **Range parameter**: Typical distance of spatial correlation
- **Nugget effect**: Measurement error + micro-scale variation

### Next Steps

1. **Visual inspection**: Load GeoTIFF files in QGIS to compare approaches
2. **Validation**: Use cross-validation to assess prediction accuracy
3. **Comparison**: Decide which approach (combined vs yearly) works better for your data
4. **Management zones**: Use interpolated surfaces to delineate high/low yield zones
5. **Temporal analysis**: Compare yearly surfaces to identify persistent patterns

### Manual Exclusion

If you identified problematic areas (e.g., harvester edge effects), go back to Section 3 and add exclusion criteria based on:
- Field names
- Coordinate ranges
- Relative yield thresholds
- Visual inspection of maps